In [1]:
import numpy as np
from pathlib import Path
import librosa
import librosa.display
from collections import defaultdict
import soundfile as sf
import tqdm
import os
from multiprocessing import Pool, cpu_count
import cv2
from matplotlib import pyplot as plt
import IPython
import pandas as pd
import scipy.signal
import time
from datetime import timedelta as td
import noisereduce as nr

from IPython.display import Audio, IFrame, display
from scipy import signal
import random
from scipy.ndimage import maximum_filter1d
import pickle
import torch

from torchlibrosa.stft import Spectrogram, LogmelFilterBank

%matplotlib inline

/usr/local/var/pyenv/versions/3.7.4/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/var/pyenv/versions/3.7.4/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/var/pyenv/versions/3.7.4/lib/python3.7/site-packages/noisereduce/noisereduce.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `

In [2]:
INPUT =  "../input/resample_sound_2"
OUTPUT = "../output/train_img_7"
    
NUM_WORKERS = cpu_count()
print(NUM_WORKERS)

12


In [4]:
sample_rate=32000
window_size=1024
hop_size=320
mel_bins=64
fmin=50
fmax=14000
window = 'hann'
center = True
pad_mode = 'reflect'
ref = 1.0
amin = 1e-10
top_db = None
        
spectrogram_extractor = Spectrogram(
            n_fft=window_size, hop_length=hop_size, 
            win_length=window_size, window=window, center=center, pad_mode=pad_mode, 
            freeze_parameters=True)
logmel_extractor = LogmelFilterBank(
            sr=sample_rate, n_fft=window_size, 
            n_mels=mel_bins, fmin=fmin, fmax=fmax, ref=ref, amin=amin, top_db=top_db, 
            freeze_parameters=True)

def audio_to_spec(audio):
    x = torch.tensor(audio).unsqueeze(0)
    x = spectrogram_extractor(x)
    x = logmel_extractor(x)
    x = x[0, 0, :, :].permute(1, 0)
    return x.numpy()

def mono_to_color(X, mean=None, std=None, norm_max=None, norm_min=None, eps=1e-6):
    # Stack X as [X,X,X]
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    X = X - mean
    std = std or X.std()
    Xstd = X / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Normalize to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

def audio2pict(path):
    x, _ = librosa.load(path=path, sr=sample_rate, mono=True)
    x_spec = audio_to_spec(x)
    img = mono_to_color(x_spec)
    cv2.imwrite(f"{OUTPUT}/{path.parent.name}/{path.name}.jpg", img)
    
def audio2vec(path):
    x, _ = librosa.load(path=path, sr=sample_rate, mono=True)
    x_spec = audio_to_spec(x)
    np.save(f"{OUTPUT}/{path.parent.name}/{path.name}", x_spec)

In [6]:
path_lst = []
for directory in Path(INPUT).iterdir():
    if directory.name in [".DS_Store"]:
        continue
    dir_paths = [f for f in directory.iterdir() if f.name not in [".DS_Store", "train_mod.csv"]]
    for dname in tqdm.tqdm_notebook(dir_paths, total=len(dir_paths)):
        file_paths = [f for f in dname.iterdir() if f.name != ".DS_Store"]
        path_lst += file_paths
        !mkdir -p "{OUTPUT}/{dname.name}"
        with Pool(NUM_WORKERS // 2) as p:
            #p.map(audio2pict, file_paths)
            p.map(audio2vec, file_paths)

/usr/local/var/pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [4]:
from PIL import Image
from pathlib import Path

for directory in Path("../output/train_img_6").iterdir():
    if directory.name == ".DS_Store":
        continue
    file_paths = [f for f in directory.iterdir() if f.name != ".DS_Store"]
    for path in file_paths:
        try:
            with open(path, 'rb') as f: img = Image.open(f)
        except:
            print(path)
            #!rm {path}